https://huggingface.co/spaces/mteb/leaderboard

https://medium.com/@thakermadhav/build-your-own-rag-with-mistral-7b-and-langchain-97d0c92fa146


In [4]:
import json
from time import sleep
import os
import sys
import asyncio
from langchain_community.document_loaders import TextLoader, PyMuPDFLoader

from langchain_community.vectorstores import FAISS,Chroma
from langchain_openai import OpenAIEmbeddings
from langchain_text_splitters import CharacterTextSplitter

from huggingface_hub import login

sys.path.append('/Users/dan/Code/LLM/llm_analyst')

from llm_analyst.documents.document import DocumentLoader
login(token = os.environ['HUGGINGFACEHUB_API_TOKEN'] )

The token has not been saved to the git credentials helper. Pass `add_to_git_credential=True` in this function directly or `--add-to-git-credential` if using via `huggingface-cli` if you want to set the git credential as well.
Token is valid (permission: read).
Your token has been saved to /Users/dan/.cache/huggingface/token
Login successful


In [5]:
# Load the json the points to all the files that will be in our document store
pmc_id = None
base_path = "/Users/dan/Code/LLM/research_data/research_for_brandan"
files_to_load_json = f"{base_path}/reference_index.json"


# Read the JSON data from the file
with open(files_to_load_json, 'r') as file:
    unique_references = json.load(file)

# Print the loaded data (or process it as needed)
print(json.dumps(unique_references, indent=4))


{
    "WBPaper00062388": {
        "title": "Gene regulatory network inference in long-lived <i>C.elegans</i> reveals modular properties that are predictive of novel aging genes.",
        "abstract": "We design a \"wisdom-of-the-crowds\" GRN inference pipeline and couple it to complex network analysis to understand the organizational principles governing gene regulation in long-lived <i>glp-1</i>/Notch <i>Caenorhabditis elegans</i>. The GRN has three layers (input, core, and output) and is topologically equivalent to bow-tie/hourglass structures prevalent among metabolic networks. To assess the functional importance of structural layers, we screened 80% of regulators and discovered 50 new aging genes, 86% with human orthologues. Genes essential for longevity-including ones involved in insulin-like signaling (ILS)-are at the core, indicating that GRN's structure is predictive of functionality. We used <i>invivo</i> reporters and a novel functional network covering 5,497 genetic interac

In [6]:
# A simple function to load the text files and add the title as meta data
async def load_doc(doc_meta, base_path):
    if doc_meta['pmcid'] != 0:
        doc_file_nm = f"{base_path}/PMC{doc_meta['pmcid']}.txt"
        url = f"https://www.ncbi.nlm.nih.gov/pmc/articles/PMC{doc_meta['pmcid']}/"
    else:
        doc_file_nm = f"{base_path}/PM{doc_meta['pmid']}.txt"
        url = f"https://pubmed.ncbi.nlm.nih.gov/{doc_meta['pmid']}/"
        
    loader = TextLoader(doc_file_nm)
    documents = loader.load()
    document = documents[0] # Expect only one doc
    document.metadata['title'] = doc_meta['title']
    document.metadata['url'] = url
    return document

    

In [48]:
# async def load_amy_docs():
#     documents = []
#     path="/Users/dan/Code/LLM/amy_papers"
#     for root, dirs, files in os.walk(path):
#             for file in files:
#                 file_path = os.path.join(root, file)
#                 loader = PyMuPDFLoader(file_path)
#                 loader_docs = loader.load()
#                 for doc in loader_docs:
#                     documents.append(doc)
#     return documents



In [49]:
# from langchain_community.embeddings import HuggingFaceHubEmbeddings
# #model = "mistralai/Mistral-7B-Instruct-v0.2"
# #model = "sentence-transformers/all-mpnet-base-v2"
# model_name = "sentence-transformers/all-MiniLM-l6-v2"
# embedding_model = HuggingFaceHubEmbeddings(
#     model=model_name,
#     task="feature-extraction",
#     huggingfacehub_api_token=os.environ['HUGGINGFACEHUB_API_TOKEN'],
# )

In [7]:
from langchain_community.embeddings import HuggingFaceEmbeddings
model_name = "sentence-transformers/all-MiniLM-l6-v2"
model_kwargs = {'device': 'mps'}
encode_kwargs = {'normalize_embeddings': False}
embedding_model = HuggingFaceEmbeddings(
    model_name=model_name,
    model_kwargs=model_kwargs,
    encode_kwargs=encode_kwargs
)

/Users/dan/miniforge3/envs/gpt-researcher/lib/python3.11/site-packages/langchain_core/_api/deprecation.py:119: LangChainDeprecationWarning: The class `HuggingFaceEmbeddings` was deprecated in LangChain 0.2.2 and will be removed in 0.3.0. An updated version of the class exists in the langchain-huggingface package and should be used instead. To use it run `pip install -U langchain-huggingface` and import as `from langchain_huggingface import HuggingFaceEmbeddings`.
  warn_deprecated(
/Users/dan/miniforge3/envs/gpt-researcher/lib/python3.11/site-packages/huggingface_hub/file_download.py:1132: FutureWarning: `resume_download` is deprecated and will be removed in version 1.0.0. Downloads always resume when possible. If you want to force a new download, use `force_download=True`.
  warnings.warn(


In [52]:
# import torch
# from transformers import AutoTokenizer, AutoModel
# from langchain.embeddings import HuggingFaceEmbeddings
# from langchain.vectorstores import FAISS,Chroma

# # Load the tokenizer and model from Hugging Face
# model_name = 'mistralai/Mistral-7B-Instruct-v0.2'
# max_length = 2048

# tokenizer = AutoTokenizer.from_pretrained(model_name)
# tokenizer.pad_token = tokenizer.eos_token
# model = AutoModel.from_pretrained(model_name)

# # Define a function to generate embeddings
# def get_embeddings(texts):
#     inputs = tokenizer(texts, return_tensors='pt',  max_length=max_length, padding=True, truncation=True)
#     with torch.no_grad():
#         outputs = model(**inputs)
#     embeddings = outputs.last_hidden_state.mean(dim=1)  # Taking the mean of the hidden states
#     return embeddings


# # Create a HuggingFaceEmbeddings class with the custom model
# class CustomHuggingFaceEmbeddings(HuggingFaceEmbeddings):
#     def embed_documents(self, texts):
#         return get_embeddings(texts).tolist()

#     def embed_query(self, text):
#         return get_embeddings([text])[0].tolist()

# # Initialize the custom embeddings model
# embedding_model = CustomHuggingFaceEmbeddings()


In [53]:
documents = await asyncio.gather(*[load_doc(doc_meta, base_path) for doc_meta in unique_references.values()])
#documents = await load_amy_docs()
text_splitter = CharacterTextSplitter(chunk_size=1000, chunk_overlap=0)
chunked_documents = text_splitter.split_documents(documents)


In [54]:
# Load chunked documents into the FAISS index
#db = await FAISS.afrom_documents(chunked_documents, embedding_model)
persist_directory = "./chroma_db"
db = await Chroma.afrom_documents(chunked_documents, embedding_model, persist_directory = persist_directory)

In [55]:
retriever_high_threshold = db.as_retriever(
    search_type="similarity_score_threshold",
    search_kwargs={'k': 6, 'score_threshold': 0.9}
)

retriever_low_threshold = db.as_retriever(
    search_type="similarity_score_threshold",
    search_kwargs={'k': 8, 'score_threshold': 0.2}
)

query = "How does DAF-19 regulate transcription of regeneration associated genes?"

# Retrieve documents with high threshold

retrieved_documents_high = retriever_high_threshold.invoke(query)
print(f"High threshold results found {len(retrieved_documents_high)}:")
for doc in retrieved_documents_high:
    print(doc)

# Retrieve documents with low threshold
retrieved_documents_low = retriever_low_threshold.invoke(query)
print(f"Low threshold results found {len(retrieved_documents_low)}:")
for doc in retrieved_documents_low:
    print(doc)


High threshold results found 0:
Low threshold results found 8:
page_content='Title:Insulin/IGF1 Signaling Inhibits Age-Dependent Axon Regeneration\nAbstract:  Summary The ability of injured axons to regenerate declines with age yet the mechanisms that regulate axon regeneration in response to age are not known. Here we show that axon regeneration in aging C. elegans motor neurons is inhibited by the conserved insulin/IGF1 receptor DAF-2. DAF-2s function in regeneration is mediated by intrinsic neuronal activity of the forkhead transcription factor DAF-16/FOXO. DAF-16 regulates regeneration independently of lifespan, indicating that neuronal aging is an intrinsic, neuron specific, and genetically regulated process. In addition, we found that daf-18/PTEN inhibits regeneration independently of age and FOXO signaling, via the TOR pathway. Finally, DLK-1, a conserved regulator of regeneration, is downregulated by insulin/IGF1 signaling, bound by DAF-16 in neurons, and is required for both D

/Users/dan/miniforge3/envs/gpt-researcher/lib/python3.11/site-packages/langchain_core/vectorstores.py:342: UserWarning: No relevant docs were retrieved using the relevance score threshold 0.9
  warnings.warn(


In [56]:
query = "How does DAF-19 regulate transcription of regeneration associated genes?"
#query = "DAF"
docs = db.similarity_search(query)
#print(docs[0].page_content)
print(len(docs))
print(docs)
for doc in docs:
    print(doc.metadata['url'])

4
[Document(page_content='Title:Insulin/IGF1 Signaling Inhibits Age-Dependent Axon Regeneration\nAbstract:  Summary The ability of injured axons to regenerate declines with age yet the mechanisms that regulate axon regeneration in response to age are not known. Here we show that axon regeneration in aging C. elegans motor neurons is inhibited by the conserved insulin/IGF1 receptor DAF-2. DAF-2s function in regeneration is mediated by intrinsic neuronal activity of the forkhead transcription factor DAF-16/FOXO. DAF-16 regulates regeneration independently of lifespan, indicating that neuronal aging is an intrinsic, neuron specific, and genetically regulated process. In addition, we found that daf-18/PTEN inhibits regeneration independently of age and FOXO signaling, via the TOR pathway. Finally, DLK-1, a conserved regulator of regeneration, is downregulated by insulin/IGF1 signaling, bound by DAF-16 in neurons, and is required for both DAF-16- and DAF-18-mediated regeneration. Together, 

### Similarity Search with score

In [57]:
docs_and_scores = db.similarity_search_with_score(query)
docs_and_scores

[(Document(page_content='Title:Insulin/IGF1 Signaling Inhibits Age-Dependent Axon Regeneration\nAbstract:  Summary The ability of injured axons to regenerate declines with age yet the mechanisms that regulate axon regeneration in response to age are not known. Here we show that axon regeneration in aging C. elegans motor neurons is inhibited by the conserved insulin/IGF1 receptor DAF-2. DAF-2s function in regeneration is mediated by intrinsic neuronal activity of the forkhead transcription factor DAF-16/FOXO. DAF-16 regulates regeneration independently of lifespan, indicating that neuronal aging is an intrinsic, neuron specific, and genetically regulated process. In addition, we found that daf-18/PTEN inhibits regeneration independently of age and FOXO signaling, via the TOR pathway. Finally, DLK-1, a conserved regulator of regeneration, is downregulated by insulin/IGF1 signaling, bound by DAF-16 in neurons, and is required for both DAF-16- and DAF-18-mediated regeneration. Together, o

### Saving and loading

In [44]:
# persist is NOT needed as the data is continually persisted 
db.persist()


/Users/dan/miniforge3/envs/gpt-researcher/lib/python3.11/site-packages/langchain_core/_api/deprecation.py:119: LangChainDeprecationWarning: Since Chroma 0.4.x the manual persistence method is no longer supported as docs are automatically persisted.
  warn_deprecated(


In [58]:
vectordb = Chroma(persist_directory=persist_directory, embedding_function=embedding_model)

In [59]:
docs = vectordb.similarity_search(query)
#print(docs[0].page_content)
print(len(docs))
print(docs)
for doc in docs:
    print(doc.metadata['url'])

4
[Document(page_content='Title:Insulin/IGF1 Signaling Inhibits Age-Dependent Axon Regeneration\nAbstract:  Summary The ability of injured axons to regenerate declines with age yet the mechanisms that regulate axon regeneration in response to age are not known. Here we show that axon regeneration in aging C. elegans motor neurons is inhibited by the conserved insulin/IGF1 receptor DAF-2. DAF-2s function in regeneration is mediated by intrinsic neuronal activity of the forkhead transcription factor DAF-16/FOXO. DAF-16 regulates regeneration independently of lifespan, indicating that neuronal aging is an intrinsic, neuron specific, and genetically regulated process. In addition, we found that daf-18/PTEN inhibits regeneration independently of age and FOXO signaling, via the TOR pathway. Finally, DLK-1, a conserved regulator of regeneration, is downregulated by insulin/IGF1 signaling, bound by DAF-16 in neurons, and is required for both DAF-16- and DAF-18-mediated regeneration. Together, 

### Merging

In [ ]:
from langchain_community.document_loaders.pubmed import PubMedLoader

# Define the PubMed query (you can specify the PubMed ID directly in the query)
query = "(22876200[UID])"

# Initialize the PubMedLoader with the query
loader = PubMedLoader(query=query)

# Load the documents matching the query
documents = loader.load()

# Check if any documents were loaded and access the full text of the first document
if documents:
    full_text = documents[0].page_content
    print("Full Document Text:")
    print(full_text)
else:
    print("No documents found for the given query.")
documents